In [1]:
import tkinter as tk
import customtkinter as ctk
import threading
import logging
from tkinter import filedialog
import cv2
import torch
import numpy as np
from PIL import Image, ImageTk
import pygame

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class DrownsinessDetector:
    def __init__(self, detection_threshold=0.85, counter_limit=20):
        self.model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp5/weights/last.pt', force_reload=True)
        self.detection_threshold = detection_threshold
        self.counter_limit = counter_limit
        self.counter = 0

    def detect_drowsiness(self, frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.model(frame)
        detections = results.xyxy[0]

        if len(detections) > 0:
            dconf = detections[0][4]
            dclass = detections[0][5]
            if dconf.item() > self.detection_threshold and dclass.item() == 16.0:  # Person class
                self.counter += 1
                logging.info(f'Drowsiness counter: {self.counter}')
                if self.counter >= self.counter_limit:
                    logging.warning('Drowsiness detected!')
                    self.counter = 0
                    return True, results.render()  # Return the rendered frame with bounding boxes
        else:
            self.counter = 0

        return False, frame  # Return the original frame without bounding boxes

class DrownsinessApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Drowsiness Detection App")
        self.geometry("800x600")

        ctk.set_appearance_mode("Dark")

        self.video_frame = ctk.CTkFrame(self, width=640, height=480)
        self.video_frame.pack(padx=10, pady=10)

        self.video_label = ctk.CTkLabel(self.video_frame)
        self.video_label.pack()

        self.config_frame = ctk.CTkFrame(self)
        self.config_frame.pack(padx=10, pady=10, fill="x")

        self.sound_file_label = ctk.CTkLabel(self.config_frame, text="Alert Sound File:")
        self.sound_file_label.pack(side="left", padx=5)

        self.sound_file_path = ctk.CTkEntry(self.config_frame)
        self.sound_file_path.pack(side="left", padx=5, fill="x", expand=True)

        self.browse_button = ctk.CTkButton(self.config_frame, text="Browse", command=self.browse_sound_file)
        self.browse_button.pack(side="left", padx=5)

        self.detector = None
        self.cap = None
        self.alert_sound = None

        self.start_button = ctk.CTkButton(self, text="Start Detection", command=self.start_detection)
        self.start_button.pack(padx=10, pady=10)

    def browse_sound_file(self):
        file_path = filedialog.askopenfilename(title="Select Sound File", filetypes=[("Audio Files", "*.wav *.mp3")])
        self.sound_file_path.delete(0, tk.END)
        self.sound_file_path.insert(0, file_path)

    def start_detection(self):
        sound_file_path = self.sound_file_path.get()

        self.detector = DrownsinessDetector()
        self.cap = cv2.VideoCapture(0)

        if sound_file_path:
            try:
                pygame.mixer.init()
                self.alert_sound = pygame.mixer.Sound(sound_file_path)
            except pygame.error as e:
                logging.error(f"Error loading sound file: {e}")
                self.alert_sound = None

        self.video_loop()

    def play_alarm(self):
        if self.alert_sound:
            self.alert_sound.play()

    def video_loop(self):
        ret, frame = self.cap.read()
        if ret:
            drowsiness_detected, processed_frame = self.detector.detect_drowsiness(frame)
            if drowsiness_detected:
                alarm_thread = threading.Thread(target=self.play_alarm)
                alarm_thread.start()

            image = Image.fromarray(processed_frame)
            photo = ImageTk.PhotoImage(image)
            self.video_label.configure(image=photo)
            self.video_label.image = photo

        self.video_label.after(10, self.video_loop)

if __name__ == "__main__":
    app = DrownsinessApp()
    app.mainloop() 

pygame 2.5.2 (SDL 2.28.3, Python 3.12.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\asus/.cache\torch\hub\master.zip
YOLOv5  2024-4-26 Python-3.12.1 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
Model summary: 157 layers, 7055974 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 
2024-04-26 11:48:36,081 - INFO - Drowsiness counter: 1
2024-04-26 11:48:36,316 - INFO - Drowsiness counter: 2
2024-04-26 11:48:36,378 - INFO - Drowsiness counter: 3
2024-04-26 11:48:36,432 - INFO - Drowsiness counter: 4
2024-04-26 11:48:36,483 - INFO - Drowsiness counter: 5
2024-04-26 11:48:36,549 - INFO - Drowsiness counter: 6
2024-04-26 11:48:36,603 - INFO - Drowsiness counter: 7
2024-04-26 11:48:36,675 - INFO - Drowsiness counter: 8
2024-04-26 11:48:36,733 - INFO - Drowsiness counter: 9
2024-04-26 11:48:36,785 - INFO - Drowsiness counter: 10
2024-04-26 11:48:36,844 - INFO - Drowsiness counter: 11
2024-04-26 11:48:36,899 - INFO - Drowsiness counter: 12
2024-04-2